In [11]:
#import matplotlib.pyplot as plt
from keras.layers import Embedding, ELU, Dropout, Flatten, Input, Dense, BatchNormalization, SpatialDropout1D
from keras.regularizers import l2, l1_l2
from keras.optimizers import Adam, Nadam
from keras.callbacks import EarlyStopping
from keras import Model
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
import pandas as pd
import numpy as np

data = pd.read_csv("train.csv",header=None,dtype='float64',names=list(range(4096))).fillna(0)
test = pd.read_csv('test.csv',header=None,dtype='float64',names=list(range(4096))).fillna(0)
label = pd.read_csv('train_label.csv')['category'].values.reshape(-1, 1)


In [12]:
maxlen=4096
embed_size=16
main_input = Input(shape=(maxlen,), dtype='float64', name='main_input')
emb = Embedding(256, embed_size, input_length=maxlen, embeddings_regularizer=l2(1e-4))(main_input)
#emb = SpatialDropout1D(0.2)(emb)
x = Flatten()(emb)
num_layers = 4
for i in range(num_layers):
    W_reg = l2(1e-4)
    if i == 0:
        W_reg = l1_l2(1e-4)
    x = Dense(10, activation='linear', kernel_regularizer=W_reg)(x) 
    x = BatchNormalization()(x) 
    x = ELU()(x)
#     if i == num_layers-1:
#         x = DeCovRegularization(0.1)(x)
    x = Dropout(0.5)(x)
    
loss_out = Dense(1, activation='sigmoid',  name='loss_out')(x)
model = Model(inputs=[main_input], outputs=[loss_out])
optimizer = Adam(lr=0.001, decay=0.5)
#optimizer = Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.5)
model.compile(optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])

es = EarlyStopping(monitor='binary_accuracy', min_delta=0, patience=0, verbose=0, mode='auto')
model.fit(data, label, batch_size=20, epochs=4, verbose=1, callbacks=[es])
result = model.predict(test, batch_size=20, verbose=1)

output = []
for item in result:
    if item >= 0.5:
        output.append(1)
    else:
        output.append(0)      
prediction = pd.DataFrame({'category': output})
prediction.to_csv('prediction.csv', index=True,header=True)


Epoch 1/4
 24380/113636 [=====>........................] - ETA: 1:40 - loss: 0.6435 - binary_accuracy: 0.7598

KeyboardInterrupt: 